# Random Forests with IMDB

### Introduction

In this lesson let's practice working with random forests on our imdb dataset.  Let's get started.

### Loading up our Data

We'll begin by loading our data.

In [27]:
import pandas as pd
url = "https://raw.githubusercontent.com/jigsawlabs-student/Random-Forests/master/imdb_movies.csv"
df = pd.read_csv(url)

In [28]:
df[:3]

,title,genre,budget,runtime,year,month,revenue
0,Avatar,Action,237000000,162.0,2009,12,2787965087
1,Pirates of the Caribbean: At World's End,Adventure,300000000,169.0,2007,5,961000000
2,Spectre,Action,245000000,148.0,2015,10,880674609


We'll select everything from `genre` through `month` as our features and revenue as our target.

In [29]:
X = df.loc[:, 'genre':'month']

In [30]:
y = df[['revenue']]

Use `any` to check that we do not have any missing values in our target values.

In [31]:
# code here

# False

revenue    False
dtype: bool

In [32]:
X[:2]
# 	genre	budget	runtime	year	month
# 0	Action	237000000	162.0	2009	12
# 1	Adventure	300000000	169.0	2007	5

,genre,budget,runtime,year,month
0,Action,237000000,162.0,2009,12
1,Adventure,300000000,169.0,2007,5


In [33]:
y[:2]

# 0    2787965087
# 1     961000000
# Name: revenue, dtype: int64

,revenue
0,2787965087
1,961000000


### Coercing our Data

Let's use a `LabelEncoder` to change our `genre` feature.  The feature also has `na`, so it is important to first change them into the string `na`.

> * Otherwise, we will see the following error -> 
> * TypeError: argument must be a string or number

Ok, go for it.

In [118]:
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

mapper = DataFrameMapper([
    (['genre'], [SimpleImputer(strategy = 'constant', fill_value = 'na'), LabelEncoder()])
], df_out = True)

In [120]:
genre_transformed = None
genre_transformed[:3]

Next replace our `genre` column in X with this data.

In [37]:
X = X.assign(genre = genre_transformed)

In [38]:
X[:2]

# 	genre	budget	runtime	year	month
# 0	0	237000000	162.0	2009	12
# 1	1	300000000	169.0	2007	5

,genre,budget,runtime,year,month
0,0,237000000,162.0,2009,12
1,1,300000000,169.0,2007,5


Next let's see if there are any other `isna` values to handle in our features.

In [39]:


# genre      False
# budget     False
# runtime    False
# year       False
# month      False
# dtype: bool

genre      False
budget     False
runtime    False
year       False
month      False
dtype: bool

Ok, and one last look at our dtypes to ensure that everything is converted to a number.

In [40]:


# genre        int64
# budget       int64
# runtime    float64
# year         int64
# month        int64
# dtype: object


genre        int64
budget       int64
runtime    float64
year         int64
month        int64
dtype: object

Ok, we are set to split our data, and train our random forest.

### Splitting our Data

Before we split our data into training, validation and test sets, we should order our data by the year and month of the movie release.  First order the X data by year and month.

In [41]:
X_sorted = None

In [42]:
X_sorted[:3]

# 	genre	budget	runtime	year	month
# 1108	2	2600000	88.0	1940	2
# 862	5	2000000	153.0	1962	6
# 1125	5	31115000	248.0	1963	6

,genre,budget,runtime,year,month
1108,2,2600000,88.0,1940,2
862,5,2000000,153.0,1962,6
1125,5,31115000,248.0,1963,6


Then we can use the indices of `X` to impose the same ordering on our series `y`.

In [43]:
y_sorted = y.loc[X_sorted.index]

In [19]:
y_sorted[:3]

# 	revenue
# 252	227817248
# 593	67112664
# 1118	51070807

,revenue
252,227817248
593,67112664
1118,51070807


Ok, now that we've ordered the data, let's split it into training, validation and test sets.  Split the data into 10 percent of the data for our validation set and 10 percent for our test set.

In [47]:
X_train.shape, X_validate.shape, X_test.shape

# ((1600, 5), (200, 5), (200, 5))

((1600, 5), (200, 5), (200, 5))

Now let's train a random forest, and evaluate it on our validation set.  Set the number of estimators equal to 100.

> Fit with `y_train.revenue` to avoid a warning.

/Users/jeff/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

First we'll evaluate the score on the training data for fun.

In [58]:
model.score(X_train, y_train)

# 0.9219605803655813

0.9219605803655813

Then evaluate the score on the validation data.

In [57]:


# 0.41940481253000517

0.41940481253000517

### Changing the Year

One way we might be able to improve the model is by removing some of the early data.  Try training different models, beginning with data from year 2000 through 2013.

We'll get you started by selecting our training set to end at 2014, and our validation set to only be movies from 2015.

In [83]:
X_train_set = X[X['year'] < 2015]
y_train = y.loc[X_train_set.index]
X_validation_set = X[X['year'] == 2015] 
y_validation = y.loc[X_validation_set.index]


Now train separate models, trying to limit the starting year to year 2000, 2001, and so on through 2013.

In [84]:
years = list(range(2000, 2014))

Add each model to the variable `models`.

In [87]:
models = []


Check the score on the validation set for each of the models, and display the year along with the corresponding scores along with the year for each of the models.

In [89]:
year_scores = None


In [122]:
years_df = None

In [123]:
years_df
# years	scores
# 0	2000	0.462312
# 1	2001	0.484643
# 2	2002	0.478579
# 3	2003	0.484120
# 4	2004	0.497659
# 5	2005	0.482426
# 6	2006	0.508346
# 7	2007	0.488352
# 8	2008	0.468401
# 9	2009	0.470982
# 10	2010	0.475468
# 11	2011	0.421079
# 12	2012	0.468214
# 13	2013	0.462303

Plot the years along with the corresponding scores on the validation set.

> Answer: <img src="./year-scores.png" width="40%">

It looks like our model peaks when we limit our data to starting in 2006.

### Hyperparameter tuning

So let's train on data 2006 and later, and reserve data from 2015 for our validation set, and data from 2016 for our test set.

The validation set is already taken care of.

In [92]:
X_validation_set = X[X['year'] == 2015] 
y_validation = y.loc[X_validation_set.index]

In [95]:
X_validation_set.year.unique()

array([2015])

Select the training set below.

In [96]:
X_post_2006 = None
y_post_2006 = None

In [97]:
X_post_2006.shape, y_post_2006.shape
# ((571, 5), (571, 1))

((571, 5), (571, 1))

Ok, now train 20 different RandomForestRegressors setting `min_samples_leaf` from 5 to 100, incrementing by 5, for a total of 20 models.  Each model should be fit with 20 estimators.

In [107]:
min_leaf_models = None

Calculate the scores for each of the models.

In [108]:
min_leaf_scores = None

Assign the number of min leafs as well as the related scores to a dataframe.  Then plot the leafs and the related scores.

In [109]:
df = pd.DataFrame({'levels': range(5, 105, 5), 'scores': depth_scores})

> Answer: <img src="leafs-scores.png" width="40%">

> The image incorrectly says levels along the x axis.

In [126]:
df[:9]

# 	leafs	scores
# 0	5	0.543086
# 1	10	0.547746
# 2	15	0.532268
# 3	20	0.502639
# 4	25	0.494886
# 5	30	0.466745
# 6	35	0.471727
# 7	40	0.465484
# 8	45	0.454884

So it looks like we have hyperparameters that peak at `min_samples_leaf` of 10.  Finally, select data from 2006 to 2015 as the training and data, and evaluate on the selected test data.

In [112]:
comb_X = None
comb_y = None

Train a RandomForestRegressor on our combined data, with min_samples_leaf equal to 10, and the number of estimators set at 300.

In [114]:
model = RandomForestRegressor(min_samples_leaf = 10, n_estimators = 300)
model.fit(comb_X, comb_y.revenue) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=10, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

Score the model on the test set.

In [116]:

# 0.562667297695517

0.562667297695517

### Summary

In this lesson, we worked through training a RandomForestRegressor.  We did so by coercing our data using the DataFrameMapper along with the LabelEncoder.  We also saw that we can treat the earliest year of our training data as a hyperparameter, and by limiting older data we can improve the accuracy of our model.  Then we moved onto working with the `min_samples_leaf` hyperparameter, which further improved our score from .50 to .56.